In [1]:
import tensorflow as tf
import os
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np

E:\Python\PY3.6\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
cwd = './17flowers/jpg'
classes = set(range(17))

In [3]:
def move_pics():
    for index,name in enumerate(classes):
        name=str(name)
        class_path = cwd+'/'+name+'/'
        save_path = './flowers_128/'+name+'/'
        if not os.path.exists(save_path):
            os.makedirs(save_path)
        for img_name in os.listdir(class_path):
            img_path = class_path+img_name

            img = Image.open(img_path)
            img = img.resize((128,128))
            img.save(save_path+img_name)

In [10]:
writer= tf.python_io.TFRecordWriter('flowers_128.tfrecords')
cwd = './flowers_128'
for index,name in enumerate(classes):
    name=str(name)
    class_path = cwd+'/'+name+'/'
    for img_name in os.listdir(class_path):
        img_path = class_path+img_name
        
        img = Image.open(img_path)

        img = img.resize((128,128))
        img_raw = img.tobytes()
        print(len(img_raw))
        example = tf.train.Example(features = tf.train.Features(feature={
            'label':tf.train.Feature(int64_list = tf.train.Int64List(value=[index])),
            'img_raw':tf.train.Feature(bytes_list=tf.train.BytesList(value=[img_raw]))
        }))
        writer.write(example.SerializeToString())
writer.close()

49152
49152
49152
49152
49152
49152
49152
49152
49152
49152
49152
49152
49152
49152
49152
49152
49152
49152
49152
49152
49152
49152
49152
49152
49152
49152
49152
49152
49152
49152
49152
49152
49152
49152
49152
49152
49152
49152
49152
49152
49152
49152
49152
49152
49152
49152
49152
49152
49152
49152
49152
49152
49152
49152
49152
49152
49152
49152
49152
49152
49152
49152
49152
49152
49152
49152
49152
49152
49152
49152
49152
49152
49152
49152
49152
49152
49152
49152
49152
49152
49152
49152
49152
49152
49152
49152
49152
49152
49152
49152
49152
49152
49152
49152
49152
49152
49152
49152
49152
49152
49152
49152
49152
49152
49152
49152
49152
49152
49152
49152
49152
49152
49152
49152
49152
49152
49152
49152
49152
49152
49152
49152
49152
49152
49152
49152
49152
49152
49152
49152
49152
49152
49152
49152
49152
49152
49152
49152
49152
49152
49152
49152
49152
49152
49152
49152
49152
49152
49152
49152
49152
49152
49152
49152
49152
49152
49152
49152
49152
49152
49152
49152
49152
49152
49152
49152
4915

In [12]:
def read_and_decode(filename):
    filename_queue = tf.train.string_input_producer([filename])
    
    reader = tf.TFRecordReader()
    _,serialized_example = reader.read(filename_queue)
    
    # 一张张读？
    features = tf.parse_single_example(serialized_example,features={
        'label':tf.FixedLenFeature([],tf.int8),
        'img_raw':tf.FixedLenFeature([],tf.string),
    })
    img = tf.decode_raw(features['img_raw'],tf.int64)
#     print(img)
    img = tf.reshape(img,[128,128,3])
    # 给你跪了，这一步超重要的！！
    img = tf.cast(img,tf.float32)*(1./255)-0.5
    label = tf.cast(features['label'],tf.int32)
    return img,label

In [34]:
tf.reset_default_graph()

filename='flowers_128.tfrecords'

filename_queue = tf.train.string_input_producer([filename])

reader = tf.TFRecordReader()
_,serialized_example = reader.read(filename_queue)

# 一张张读？
features = tf.parse_single_example(serialized_example,features={
    'label':tf.FixedLenFeature([],tf.int64),
    'img_raw':tf.FixedLenFeature([],tf.string),
})
# 这个是傻逼吧…用int64解码，应该用int8解码啊………
img1 = tf.decode_raw(features['img_raw'],tf.int8) 

img = tf.reshape(img1,[128,128,3])
# 给你跪了，这一步超重要的！！
img = tf.cast(img,tf.float32)*(1./255)-0.5
label = tf.cast(features['label'],tf.int32)

with tf.Session() as sess:
    init_op = tf.global_variables_initializer()
    sess.run(init_op)
    
    # 这个是啥？
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(coord=coord)

    
        

    
    for i in range(1):
        example, l =sess.run([img,label])

        img = Image.fromarray(example,'RGB')
        print(example,l)
    coord.request_stop()
    coord.join(threads)

[[[-0.57843137 -0.5588235  -0.5470588 ]
  [-0.5470588  -0.527451   -0.5156863 ]
  [-0.5627451  -0.5470588  -0.5352941 ]
  ...
  [-0.40588236 -0.4254902  -0.44117647]
  [-0.4254902  -0.4490196  -0.44117647]
  [-0.29215685 -0.3117647  -0.2960784 ]]

 [[-0.6607843  -0.64117646 -0.62941176]
  [-0.70000005 -0.68039215 -0.66862744]
  [-0.6254902  -0.6098039  -0.5980392 ]
  ...
  [-0.36666667 -0.38627452 -0.4019608 ]
  [-0.39411765 -0.41764706 -0.40980393]
  [-0.22549018 -0.24509802 -0.22941175]]

 [[-0.64509803 -0.6137255  -0.60588235]
  [-0.80980396 -0.79019606 -0.7784314 ]
  [-0.6137255  -0.5980392  -0.5862745 ]
  ...
  [-0.3745098  -0.39411765 -0.40980393]
  [-0.42941177 -0.45294118 -0.44509804]
  [-0.23333332 -0.25294116 -0.23725489]]

 ...

 [[-0.98627454 -0.954902   -0.28823528]
  [-0.14313725 -0.15098038 -0.30392155]
  [-0.272549   -0.30784312 -0.36666667]
  ...
  [-0.31960785 -0.2960784  -0.34313726]
  [-0.3392157  -0.3156863  -0.3627451 ]
  [-0.31960785 -0.2960784  -0.34313726]]

 [

In [ ]:
tf.train.Coordinator??

In [31]:
tf.

Object `tf.fromstring` not found.
